<div style="background-color:green; color:white; padding:10px; font-size:20px">
1 - Setup

In [1]:
!pip install boto3

  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [2]:
import random

import boto3
import data_handling
import pandas as pd
import time

# Project Scripts
from data_handling import view_rand_df

<div style="background-color:teal; color:white; padding:0px; font-size:20px">
♺ Refresh

In [3]:
# Re-freshing the module import for development purposes
from importlib import reload
reload(data_handling)

<module 'data_handling' from '/root/projects/behavior_project/data_handling.py'>

#### Sagemaker User & IAM Credentials

In [4]:
# Create credentials session (log-in to AWS account)
sess = boto3.Session()
# Get information about current User
sts = sess.client("sts")
response = sts.get_caller_identity()

print("Your AWS Account:  717145514721")
print("Logged-in Account:", response["Account"])
print("User ARN:", response["Arn"])

Your AWS Account:  717145514721
Logged-in Account: 717145514721
User ARN: arn:aws:sts::717145514721:assumed-role/AWS-SageMakerFullAccess-Role/SageMaker


### S3 Connection

In [5]:
import boto3

# Create a session using your AWS credentials
s3 = boto3.client("s3")

# List all bucket names
response = s3.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket["Name"] for bucket in response["Buckets"]]

# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['behavior-project', 'pytorch-aws-train-deploy-project', 'sagemaker-us-east-1-717145514721']


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


<div style="background-color:green; color:white; padding:10px; font-size:20px">
2 - Data Import

In [6]:
from data_handling import load_to_dict
from data_handling import optimize_dataframe_dtypes
from data_handling import concatenate_csvs_from_s3
from io import StringIO

<div style="background-color:darkseagreen; color:black; padding:0px; font-size:20px">
trace_df_dict    --->    trace_df

#### mPFC

In [7]:
bucket_name = 'behavior-project'
prefix = 'processed-data/processed-data/trace-df-dictionaries/mpfc-icss/'

mPFC_trace_df = concatenate_csvs_from_s3(
    bucket_name=bucket_name,
    prefix=prefix)

In [8]:
mPFC_trace_df.drop(columns=['session_zone_(h)','session_zone_(d)', 'session_zone_(v)', 'session_zone_(t)'], inplace=True)
mPFC_trace_df['session_id'] = mPFC_trace_df['session_id'].astype(str)
mPFC_trace_df.head(3)

,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,trace_num,session_id
0,1970-01-01 00:00:10.340999603,143.0,163.0,True,False,False,False,0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON
1,1970-01-01 00:00:10.373999595,143.0,163.0,True,False,False,False,0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON
2,1970-01-01 00:00:10.440999984,143.0,163.0,True,False,False,False,0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON


#### FR

In [9]:
bucket_name = 'behavior-project'
prefix = 'processed-data/processed-data/trace-df-dictionaries/fr-icss/'

FR_trace_df = concatenate_csvs_from_s3(
    bucket_name=bucket_name,
    prefix=prefix)

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [10]:
FR_trace_df.drop(columns=['session_zone_(h)','session_zone_(d)', 'session_zone_(v)', 'session_zone_(t)'], inplace=True)
FR_trace_df['session_id'] = FR_trace_df['session_id'].astype(str)
FR_trace_df.head(3)

,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,trace_num,session_id
0,1970-01-01 00:00:45.854000091,448.0,383.0,False,False,True,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON
1,1970-01-01 00:00:45.963001251,448.0,382.0,False,False,True,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON
2,1970-01-01 00:00:46.076000213,448.0,382.0,False,False,True,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON


<div style="background-color:green; color:white; padding:10px; font-size:20px">
3 - Zone Entry

In [11]:
from data_handling import tally_transitions

#### mPFC

In [12]:
DF = mPFC_trace_df
T_COL_IDX = 3
D_COL_IDX = 4
H_COL_IDX = 5
V_COL_IDX = 6

mPFC_zone_entries = tally_transitions(df = DF,
                                       T_COL_IDX = T_COL_IDX,
                                       D_COL_IDX = D_COL_IDX,
                                       H_COL_IDX = H_COL_IDX,
                                       V_COL_IDX = V_COL_IDX)
mPFC_zone_entries.head(3)

,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,trace_num,session_id,t_total,d_total,h_total,v_total
0,1970-01-01 00:00:10.340999603,143.0,163.0,True,False,False,False,0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,0,0,0,0
1,1970-01-01 00:00:10.373999595,143.0,163.0,True,False,False,False,0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,0,0,0,0
2,1970-01-01 00:00:10.440999984,143.0,163.0,True,False,False,False,0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,0,0,0,0


#### FR

In [13]:
DF = FR_trace_df
T_COL_IDX = 3
D_COL_IDX = 4
H_COL_IDX = 5
V_COL_IDX = 6

FR_zone_entries = tally_transitions(df = DF,
                                       T_COL_IDX = T_COL_IDX,
                                       D_COL_IDX = D_COL_IDX,
                                       H_COL_IDX = H_COL_IDX,
                                       V_COL_IDX = V_COL_IDX)
FR_zone_entries.head(3)

,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,trace_num,session_id,t_total,d_total,h_total,v_total
0,1970-01-01 00:00:45.854000091,448.0,383.0,False,False,True,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,0,0,0,0
1,1970-01-01 00:00:45.963001251,448.0,382.0,False,False,True,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,0,0,0,0
2,1970-01-01 00:00:46.076000213,448.0,382.0,False,False,True,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,0,0,0,0


<div style="background-color:green; color:white; padding:10px; font-size:20px">
4 - Trace Distance & Area

In [49]:
from data_handling import calculate_trace_length, calculate_area_span

#### mPFC

In [40]:
DF = mPFC_trace_df
TIME_COL_IDX = 0
X_COL_IDX = 1
Y_COL_IDX = 2
TRACE_NUM_IDX = 7
SESSION_ID_IDX = 8

mPFC_trace_len = calculate_trace_length(df = DF,
                                         time_col_idx = TIME_COL_IDX,
                                         x_col_idx = X_COL_IDX,
                                         y_col_idx = Y_COL_IDX,
                                         trace_num_idx = TRACE_NUM_IDX,
                                         session_id_idx = SESSION_ID_IDX)

mPFC_trace_span = calculate_area_span(df = DF,
                                         x_col_idx = X_COL_IDX,
                                         y_col_idx = Y_COL_IDX,
                                         trace_num_idx = TRACE_NUM_IDX,
                                         session_id_idx = SESSION_ID_IDX)

mPFC_len_span_df = pd.merge(mPFC_trace_len, mPFC_trace_span, on=['session_id', 'trace_num'])
mPFC_len_span_df.head(3)

,session_id,trace_num,trace_length,xy_area_span
0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,0,190.933121,60.27
1,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,1,211.826843,73.10
2,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,2,380.396942,363.12


#### FR

In [50]:
DF = FR_trace_df
TIME_COL_IDX = 0
X_COL_IDX = 1
Y_COL_IDX = 2
TRACE_NUM_IDX = 7
SESSION_ID_IDX = 8

FR_trace_len = calculate_trace_length(df = DF,
                                         time_col_idx = TIME_COL_IDX,
                                         x_col_idx = X_COL_IDX,
                                         y_col_idx = Y_COL_IDX,
                                         trace_num_idx = TRACE_NUM_IDX,
                                         session_id_idx = SESSION_ID_IDX)

FR_trace_span = calculate_area_span(df = DF,
                                         x_col_idx = X_COL_IDX,
                                         y_col_idx = Y_COL_IDX,
                                         trace_num_idx = TRACE_NUM_IDX,
                                         session_id_idx = SESSION_ID_IDX)

FR_len_span_df = pd.merge(FR_trace_len, FR_trace_span, on=['session_id', 'trace_num'])
FR_len_span_df.head(3)

,session_id,trace_num,trace_length,xy_area_span
0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,0,374.319794,503.70
1,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,1,347.604828,140.42
2,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,2,342.018097,178.88


<div style="background-color:green; color:white; padding:10px; font-size:20px">
5 - Save

#### mPFC

In [14]:
FILE = mPFC_zone_entries
S3_DIR_PATH = "s3://behavior-project/processed-data/processed-data/zone-entry-dfs/"
FILE_NAME = 'mpfc-zone-entries-df.csv'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)
file_path = f"{prefix}/{FILE_NAME}"

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
FILE.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=file_path, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'TP1790GPYGH821GV',
  'HostId': 'GAzL/tlMk5q1NCtwpgl5MOrXMow6UQnX2aTQM6dXSDWHqd1XlJrTSzBuw5Z+ofP6AcL0Qr1ZMJE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'GAzL/tlMk5q1NCtwpgl5MOrXMow6UQnX2aTQM6dXSDWHqd1XlJrTSzBuw5Z+ofP6AcL0Qr1ZMJE=',
   'x-amz-request-id': 'TP1790GPYGH821GV',
   'date': 'Wed, 23 Aug 2023 16:22:38 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"77e4ae88dfc9681aa905156faeb6cf63"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"77e4ae88dfc9681aa905156faeb6cf63"',
 'ServerSideEncryption': 'AES256'}

In [53]:
FILE = mPFC_len_span_df
S3_DIR_PATH = "s3://behavior-project/processed-data/processed-data/trace-metric-dfs/"
FILE_NAME = 'mpfc-len-span-df.csv'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)
file_path = f"{prefix}/{FILE_NAME}"

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
FILE.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=file_path, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '3CNNQR1DFNXZDR1T',
  'HostId': 'JP/Wk4sHesF2vkCVclXrcaFSLbUFWar9a15oMq19gcGtMVObYxyaYo4f6+p3Tv+DReZ3WVtpmrw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'JP/Wk4sHesF2vkCVclXrcaFSLbUFWar9a15oMq19gcGtMVObYxyaYo4f6+p3Tv+DReZ3WVtpmrw=',
   'x-amz-request-id': '3CNNQR1DFNXZDR1T',
   'date': 'Wed, 09 Aug 2023 19:03:18 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"6664cf6c6cf1acfde9e9a4ee6174ab01"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"6664cf6c6cf1acfde9e9a4ee6174ab01"',
 'ServerSideEncryption': 'AES256'}

#### FR

In [15]:
FILE = FR_zone_entries
S3_DIR_PATH = "s3://behavior-project/processed-data/processed-data/zone-entry-dfs/"
FILE_NAME = 'fr-zone-entries-df.csv'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)
file_path = f"{prefix}/{FILE_NAME}"

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
FILE.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=file_path, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '9EWDVH5WVZ5295XG',
  'HostId': 'd+Xukyhq/kfW4tUrsUxm3gVPDroWdOwbwYzgL2754Lvk695qCuyFxmAujtPgcjL9vVDiQ30ueFE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'd+Xukyhq/kfW4tUrsUxm3gVPDroWdOwbwYzgL2754Lvk695qCuyFxmAujtPgcjL9vVDiQ30ueFE=',
   'x-amz-request-id': '9EWDVH5WVZ5295XG',
   'date': 'Wed, 23 Aug 2023 16:22:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f3259faf3583115f1551c137a64f0f6d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f3259faf3583115f1551c137a64f0f6d"',
 'ServerSideEncryption': 'AES256'}

In [55]:
FILE = FR_len_span_df
S3_DIR_PATH = "s3://behavior-project/processed-data/processed-data/trace-metric-dfs/"
FILE_NAME = 'fr-len-span-df.csv'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)
file_path = f"{prefix}/{FILE_NAME}"

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
FILE.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=file_path, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'H7PV1C14X1Y42TEJ',
  'HostId': '3aKBcpM6db4mgnCsM7mmGqO3dM2c1Yt83TofSUF48ag7bTJFLoLFNcrwZLVbM8AH7UE7rE0krIM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3aKBcpM6db4mgnCsM7mmGqO3dM2c1Yt83TofSUF48ag7bTJFLoLFNcrwZLVbM8AH7UE7rE0krIM=',
   'x-amz-request-id': 'H7PV1C14X1Y42TEJ',
   'date': 'Wed, 09 Aug 2023 19:03:23 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"ed153abe3ace024f4de0ef10e9bb4d31"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"ed153abe3ace024f4de0ef10e9bb4d31"',
 'ServerSideEncryption': 'AES256'}